In [1]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.utils import to_categorical
import numpy as np
import math
import os
import pandas as pd

check = True 


# load data 

# 개인 아이디어 : 주장을 넣었을 때, ChatGPT 에 근거를 넣기
# 현재 모델은 어떤 단어가 들어갔는지만 고려. 깊게 발전시키지 않음. 

# 데이터를 보니, 의견을 알리는 것도 있음. 즉, 사실 주장을 증명하기엔 애매모호한 주제들이 많음. 
# 데이터셋도 필터링이 필요할 듯. 흐으음.. 한편으론 글의 타입이 변하기만 해도 애매한 부분이


# load tokenize 
# make model & train 
# 학습한 모델 불러오기 -> 모델이 상당히 조잡한데... 


csv_location = f"C:/Users/user/Desktop/test_ethic/20231110_total.csv"


def to_one_hot(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

df = pd.read_csv(csv_location, encoding='CP949')

def embedding(data, max_words, word_index = None) : 
    tokenizer = Tokenizer(num_words=max_words)
    # Tokenizer 내부의 단어-인덱스 매핑을 구축함. 
    tokenizer.fit_on_texts(data)

    # 단어 - 인덱스 매핑 저장하기 
    if word_index is None : 
        new_word_index = tokenizer.word_index
    else : 
        tokenizer.word_index = word_index
    

    # texts 에 담겨있는 텍스트 데이터를 정수 시퀀스로 변환
    train_data = tokenizer.texts_to_sequences(data) 
    if word_index is None : 
        return new_word_index, train_data 
    else : 
        return word_index, train_data



In [2]:
#def load_data(csv_location) : 
check = True 
if check == True : 
    df = pd.read_csv(csv_location, encoding='CP949')
    #df['label'] = df['label'].replace({'판단 유보' : -1, '사실' : 2, '대체로 사실': 2, '대체로 사실 아님' : 0, '전혀 사실 아님' : 0, '논쟁 중' : -1, '절반의 사실' : 1, '판단 유보' : 1})
    df['label'] = df['label'].replace({'판단 유보' : 1, '사실' : 2, '대체로 사실': 2, '대체로 사실 아님' : 0, '전혀 사실 아님' : 0, '논쟁 중' : 1, '절반의 사실' : 1, '판단 유보' : 1})


    filtered_df = df[df['label'] != -1]

    train_texts = filtered_df["제목"].tolist()

    train_label = filtered_df['label'].tolist()
    
    train_label = np.asarray(train_label).astype('float32')
    test_ratio = math.floor(len(train_texts) * 0.2)

    max_words = 1000
    maxlen = 1000

    word_index, train_data = embedding(data = train_texts, max_words=max_words)    
    ont_hot_train_data = to_one_hot(train_data, dimension=max_words)
    
    x_train = ont_hot_train_data[test_ratio:]
    y_train = train_label[test_ratio:]
    y_train_one_hot = to_categorical(y_train, num_classes=3)


    x_val = ont_hot_train_data[:test_ratio]
    y_val = train_label[:test_ratio]
    y_val_one_hot = to_categorical(y_val, num_classes=3)

In [3]:
import json

# word_index 정보 저장
with open('word_index.json', 'w') as file:
    json.dump(word_index, file)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.optimizers import Adam


import numpy as np

if check == True : 

    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=50, input_length=maxlen))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(3, activation='softmax'))

    custom_optimizer = Adam(learning_rate=0.01)
    model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    history = model.fit(x_train, y_train_one_hot, epochs=50, batch_size=128,  validation_data=(x_val, y_val_one_hot))
    
    model.save('test.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 50)          50000     
                                                                 
 flatten (Flatten)           (None, 50000)             0         
                                                                 
 dense (Dense)               (None, 64)                3200064   
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 3,250,259
Trainable params: 3,250,259
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
30/30 [==============================] - 2s 70ms/step - loss: 2.2710 - accuracy: 0.4769 - val_loss: 1.0862 - val_accuracy: 0.3003
Epoch 2/50
30/30 [===================

In [7]:
from tensorflow.keras.models import load_model
import pandas as pd
import random 
import json

# 모델 로드
model = load_model('test.h5')
with open('word_index.json', 'r') as file:
    loaded_word_index = json.load(file)


test_csv_location = "../Scraper output/output_20231123_161916.csv"

#test_df = pd.read_csv(test_csv_location, encoding='CP949')
test_df = pd.read_csv(test_csv_location, encoding='CP949')
test_data = test_df['Title'].tolist()

word_index, embed_data = embedding(data = test_data, max_words=max_words, word_index=word_index)    
ont_hot_data = to_one_hot(embed_data, dimension=max_words)


# 새로운 테스트 데이터셋에 대한 예측
predictions = model.predict(ont_hot_data)  # new_x_test는 새로운 테스트 데이터셋
max_indices = np.argmax(predictions, axis=1)

# x_val 부분은 사전 전처리 코드를 추가해둬야겠네. 
# embedding 전의 데이터와 구분할 필요가 있겠다. 

#temporary_category = ['조선일보' for index in max_indices]
#categories = ['JTBC', 'KBS', 'MBC', 'SBS']
#temporary_category = [random.choice(categories) for _ in max_indices]

Network = test_df['Network'].tolist()

# 예측 결과와 테스트 데이터를 DataFrame으로 만들기

label_values = ['거짓', '논란', '사실']  # 실제 레이블에 맞게 수정
mapped_labels = [label_values[prediction] for prediction in max_indices]






32/32 [==============================] - 0s 4ms/step


In [ ]:
from datetime import datetime

def convert_date_time_format(date_str):
    # 문자열의 날짜 및 시간 형식을 파싱
    parsed_date = None
    
    # 각 날짜 및 시간 형식에 대한 포맷 시도
    formats_to_try = [
        "%Y-%m-%d %H:%M:%S",           # 2023-11-14 11:08:04
        "%Y-%m-%d %I:%M:%S %p",        # 2023-11-15 02:31:01
        "%Y. %m. %d. %p %I:%M",        
        "%Y.%m.%d. 오후 %I:%M",        # 2023.11.15. 오후 1:53
        "%Y.%m.%d. 오전 %I:%M",        # 2023.11.13. 오전 7:48
    
    ]
    for date_format in formats_to_try:
        try:
            parsed_date = datetime.strptime(date_str, date_format)
            break
        except ValueError:
            continue
    
    if parsed_date is None:
        raise ValueError(f"날짜 형식을 인식할 수 없습니다: {date_str}")

    # 새로운 형식으로 변환
    formatted_date = parsed_date.strftime("%Y.%m.%d %p %I:%M")

    return formatted_date

# 변환된 날짜 데이터
converted_dates = [convert_date_time_format(date_str) for date_str in date_data]

# 결과 출력
for original, converted in zip(date_data, converted_dates):
    print(f"Original: {original} -> Converted: {converted}")


Original: 2023-11-15 08:49:03 -> Converted: 2023.11.15 AM 08:49
Original: 2023-11-14 12:57:30 -> Converted: 2023.11.14 PM 12:57
Original: 2023-11-15 17:00:00 -> Converted: 2023.11.15 PM 05:00
Original: 2023-11-15 19:02:01 -> Converted: 2023.11.15 PM 07:02
Original: 2023-11-15 17:26:01 -> Converted: 2023.11.15 PM 05:26
Original: 2023-11-15 17:41:03 -> Converted: 2023.11.15 PM 05:41
Original: 2023-11-14 15:41:01 -> Converted: 2023.11.14 PM 03:41
Original: 2023-11-14 11:06:19 -> Converted: 2023.11.14 AM 11:06
Original: 2023-11-14 09:34:01 -> Converted: 2023.11.14 AM 09:34
Original: 2023-11-14 11:21:03 -> Converted: 2023.11.14 AM 11:21
Original: 2023-11-15 13:52:01 -> Converted: 2023.11.15 PM 01:52
Original: 2023-11-14 06:41:34 -> Converted: 2023.11.14 AM 06:41
Original: 2023-11-14 19:15:01 -> Converted: 2023.11.14 PM 07:15
Original: 2023-11-14 17:09:04 -> Converted: 2023.11.14 PM 05:09
Original: 2023-11-14 11:13:01 -> Converted: 2023.11.14 AM 11:13
Original: 2023-11-15 04:03:27 -> Convert

In [17]:
result_df = pd.DataFrame({'date' : test_df['Time'], 'article title': test_data, '출판사' : Network , 'link' : test_df['URL'].tolist(), 'result': mapped_labels})

upper_network = ['YTN','머니투데이', '연합뉴스', 'SBS', 'TV조선', '한겨레', '오마이뉴스', '노컷뉴스', 'MBC', '이데일리'] 

condition = result_df['출판사'].isin(upper_network)

filtered_result_df = result_df[condition]


In [13]:
count_network = [] 
for n in list(set(Network)) : 
    num = len(result_df[result_df['출판사'] == n])
    count_network.append([n, num])

In [16]:
count_network

[['YTN', 126],
 ['머니투데이', 120],
 ['연합뉴스', 108],
 ['SBS', 94],
 ['TV조선', 63],
 ['한겨레', 62],
 ['오마이뉴스', 62],
 ['노컷뉴스', 55],
 ['MBC', 37],
 ['이데일리', 33],
 ['더팩트', 28],
 ['MBN', 27],
 ['JTBC', 26],
 ['매일경제', 24],
 ['세계일보', 23],
 ['KBS', 20],
 ['뉴스1', 17],
 ['한국일보', 16],
 ['파이낸셜뉴스', 14],
 ['문화일보', 12],
 ['경향신문', 10],
 ['아시아경제', 7],
 ['뉴시스', 7],
 ['중앙일보', 6],
 ['머니S', 5],
 ['한국경제', 5],
 ['아이뉴스24', 5],
 ['서울신문', 3],
 ['디지털타임스', 2],
 ['국민일보', 1],
 ['조선비즈', 1]]

In [18]:
filtered_result_df

,date,article title,출판사,link,result
0,2023.11.22,[따져보니] 오픈AI 사태…'기술-규제' 전쟁?,TV조선,https://n.news.naver.com/article/448/0000437297,거짓
2,2023.11.22,[팩트체크] 소방관이 지급받는 간병료가 간병인 일당도 안된다?,이데일리,https://n.news.naver.com/article/018/0005625128,거짓
4,2023.11.21,[따져보니] 구급차 출동 요청…어디까지 허용?,TV조선,https://n.news.naver.com/article/448/0000437095,거짓
5,2023.11.21,"[사실은] ""사이렌 오더는 취소 안 돼요""…문제 없나 보니",SBS,https://n.news.naver.com/article/055/0001107818,거짓
6,2023.11.21,"""서울 편입 찬성 2배 늘었다?"" 김포시 여론조사의 함정 [오마이팩트]",오마이뉴스,https://n.news.naver.com/article/047/0002413399,거짓
...,...,...,...,...,...
1014,2020.03.26.,[팩트체크] 조주빈이 협조 안하면 수십억 가상화폐 몰수 못한다?,머니투데이,https://n.news.naver.com/article/008/0004384550,거짓
1015,2020.03.24.,[팩트체크]'조국'이 'n번방·박사방' 피의자 신상공개·포토라인 막았다?,머니투데이,https://n.news.naver.com/article/008/0004384114,거짓
1016,2020.03.23.,[팩트체크] 택배 만져도 코로나 감염될까,머니투데이,https://n.news.naver.com/article/008/0004382258,거짓
1017,2020.03.22.,[팩트체크]'n번방·박사방'가입자 수만명 처벌 못하는 이유는?,머니투데이,https://n.news.naver.com/article/008/0004381118,거짓


In [ ]:
# Softmax 함수를 사용하여 확률 분포 얻기
probabilities = model.predict(ont_hot_train_data, verbose=0)

# 각 예측에서 가장 높은 확률을 가진 클래스의 인덱스 가져오기
predicted_labels = np.argmax(probabilities, axis=1)

# 예측된 레이블을 실제 레이블 값으로 변환 (예: 0부터 시작하는 클래스 레이블)
# 여기서는 가정적으로 label_values를 정의하여 사용합니다. 실제로는 데이터셋에 따라 다르게 설정될 수 있습니다.


In [19]:
filtered_result_df.to_csv(f"total_result.csv", encoding='utf-8-sig', index=False)

In [23]:
category = list(set(filtered_result_df['출판사'].tolist()))

info = []
for c in category : 
    filtered_df = filtered_result_df[filtered_result_df['출판사'] == c]
    num_data = len(filtered_df)
    num_fact = len(filtered_df[filtered_df['result'] == '사실'])
    num_conv = len(filtered_df[filtered_df['result'] == '논란'])
    num_fake = len(filtered_df[filtered_df['result'] == '거짓'])

    info.append([c, num_data, num_fact, num_conv, num_fake])


columns = ['Category', 'Num_Data', 'Num_Fact', 'Num_Conv', 'Num_Fake']
df_info = pd.DataFrame(info, columns=columns)

print(df_info)


  Category  Num_Data  Num_Fact  Num_Conv  Num_Fake
0     TV조선        63         0         0        63
1      MBC        37         0         0        37
2     이데일리        33         0         0        33
3      한겨레        62         0         0        62
4      SBS        94         0         0        94
5     노컷뉴스        55         0         0        55
6    머니투데이       120         0         0       120
7    오마이뉴스        62         0         0        62
8     연합뉴스       108         0         0       108
9      YTN       126         0         0       126


In [25]:
df_info.to_csv(f"result_per_category.csv", encoding='utf-8-sig', index=False)